In [1]:
# Welcome to the OL316-parsing tool
# This tool has been created to convert the avaliable OL316 reports into a machine readable dataset
# For any question, Please contact Dr. El-Yabroudi, email: melyabrou@ltu.edu 

In [ ]:
#imports
import numpy as np 
import csv   
import fitz
import pandas as pd
from os import listdir
from os.path import isfile, join
import os
from IPython.display import Image, display
import requests

In [2]:
#Paths
#path to the reports directoris 
pdf_reports_dirs = r'C:/Users/melyabrou/Downloads/crashes_analysis/dataset/dataset/reports_pdf/'

# Path to save CSV file (machine readable dataset)
csv_file = r'C:/Users/melyabrou/Downloads/ol316.csv'

In [3]:
#functions
def extract_form_data_from_pdf(pdf_path):
    # Open the PDF
    pdf_document = fitz.open(pdf_path)
    
    # List to hold form field data
    data = []
    fields_names = []
    fields_values = []
    
    # Loop through pages to find form fields
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        field_values = page.widgets()  # Extract widgets (form fields)
        #print (field_values)

        for field in field_values:
            
            field_name = field.field_name if field.field_name else "Unnamed Field"
            field_value = field.field_value  if field.field_value  else "Empty"

            fields_names.append(field_name)
            fields_values.append(field_value)
            
            # Handle checkboxes
            if field.field_type == fitz.PDF_WIDGET_TYPE_CHECKBOX:
                field_value = "Checked" if field.field_value == 'Yes' else "Unchecked"
            

    #now preparing the in row format for the intresting fields
    data.append(fields_values[fields_names.index('MANufACTuRERS NAME')])
    data.append(fields_values[fields_names.index('BuSINESS NAME')])
    data.append(fields_values[fields_names.index('DATE Of ACCIDENT')])
    data.append(fields_values[fields_names.index('Time of Accident')])

    if(fields_values[fields_names.index('AM')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('PM')] != 'Empty'):
        data.append('1')
    else:
        data.append('-1')

    data.append(fields_values[fields_names.index('VEhICLE YEAR')])
    data.append(fields_values[fields_names.index('MAkE')])
    data.append(fields_values[fields_names.index('MODEL')])
    data.append(fields_values[fields_names.index('STATE VEhICLE IS REGISTERED IN')])

    data.append(fields_values[fields_names.index('section 2  accident infoRmation.0')])
    data.append(fields_values[fields_names.index('section 2  accident infoRmation.1.0')])
    data.append(fields_values[fields_names.index('section 2  accident infoRmation.1.1.0')])
    data.append(fields_values[fields_names.index('section 2  accident infoRmation.1.1.1.0')])
    data.append(fields_values[fields_names.index('section 2  accident infoRmation.1.1.1.1')])
    
    if(fields_values[fields_names.index('Moving')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('Stopped in Traffic')] != 'Empty'):
        data.append('1')
    else:
        data.append('-1')


    if(fields_values[fields_names.index('Pedestrian')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('Bicyclist')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('Other')] != 'Empty'):
        data.append('2')
        
    else:
        data.append('-1')


    data.append(fields_values[fields_names.index('NuMBER Of VEhICLES INVOLVED')])

    
    if(fields_values[fields_names.index('Unknown')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('None')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('minor')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('Moderate')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('major')] != 'Empty'):
        data.append('4')
    else:
        data.append('-1')


    #damage_info = []
    damage_info = ''
    if(fields_values[fields_names.index('Left Rear 1')] != 'Empty'):
        #damage_info.append('0')
        damage_info = damage_info + '0 '
    if(fields_values[fields_names.index('Rear Bumper')] != 'Empty'):
        damage_info = damage_info + '1 '
    if(fields_values[fields_names.index('Right Rear 1')] != 'Empty'):
        damage_info = damage_info + '2 '
    if(fields_values[fields_names.index('Left Rear 2')] != 'Empty'):
        damage_info = damage_info + '3 '
    if(fields_values[fields_names.index('Left Rear 3')] != 'Empty'):
        damage_info = damage_info + '4 '

    if(fields_values[fields_names.index('Right Rear 2')] != 'Empty'):
        damage_info = damage_info + '5 '
    if(fields_values[fields_names.index('Right Rear 3')] != 'Empty'):
        damage_info = damage_info + '6 '
    if(fields_values[fields_names.index('Left Rear Passenger 1')] != 'Empty'):
        damage_info = damage_info + '7 '
    if(fields_values[fields_names.index('Left Rear Passenger 2')] != 'Empty'):
        damage_info = damage_info + '8 '
    if(fields_values[fields_names.index('Right Rear Passenger 1')] != 'Empty'):
        damage_info = damage_info + '9 '

    if(fields_values[fields_names.index('Right Rear Passenger 2')] != 'Empty'):
        damage_info = damage_info + '10 '
    if(fields_values[fields_names.index('Left Rear Passenger 3')] != 'Empty'):
        damage_info = damage_info + '11 '
    if(fields_values[fields_names.index('Left Rear Passenger 4')] != 'Empty'):
        damage_info = damage_info + '12 '
    if(fields_values[fields_names.index('Right Rear Passenger 3')] != 'Empty'):
        damage_info = damage_info + '13 '
    if(fields_values[fields_names.index('Right Rear Passenger 4')] != 'Empty'):
        damage_info = damage_info + '14 '

    if(fields_values[fields_names.index('Front Driver Side 1')] != 'Empty'):
        damage_info = damage_info + '15 '
    if(fields_values[fields_names.index('Front Driver Side 2')] != 'Empty'):
        damage_info = damage_info + '16 '
    if(fields_values[fields_names.index('Front Passenger Side 1')] != 'Empty'):
        damage_info = damage_info + '17 '
    if(fields_values[fields_names.index('Front Passenger Side 2')] != 'Empty'):
        damage_info = damage_info + '18 '
    if(fields_values[fields_names.index('Front Driver Side 3')] != 'Empty'):
        damage_info = damage_info + '19 '

    if(fields_values[fields_names.index('Front Driver Side 4')] != 'Empty'):
        damage_info = damage_info + '20 '
    if(fields_values[fields_names.index('Front Passenger Side 3')] != 'Empty'):
        damage_info = damage_info + '21 '
    if(fields_values[fields_names.index('Front Passenger Side 4')] != 'Empty'):
        damage_info = damage_info + '22 '
    if(fields_values[fields_names.index('Left Front Corner 1')] != 'Empty'):
        damage_info = damage_info + '23 '
    if(fields_values[fields_names.index('Left Front Corner 2')] != 'Empty'):
        damage_info = damage_info + '24 '

    if(fields_values[fields_names.index('Right Front Corner 1')] != 'Empty'):
        damage_info = damage_info + '25 '
    if(fields_values[fields_names.index('Right Front Corner 2')] != 'Empty'):
        damage_info = damage_info + '26 '
    if(fields_values[fields_names.index('Left Front Corner 3')] != 'Empty'):
        damage_info = damage_info + '27 '
    if(fields_values[fields_names.index('Front Bumper')] != 'Empty'):
        damage_info = damage_info + '28 '
    if(fields_values[fields_names.index('Right Front Corner 3')] != 'Empty'):
        damage_info = damage_info + '29 '

    if(len(damage_info) == 0):
        damage_info = damage_info + '-1'
    
    data.append(damage_info)
    
    data.append(fields_values[fields_names.index('VEhICLE YEAR_2')])
    data.append(fields_values[fields_names.index('MODEL_2')])
    data.append(fields_values[fields_names.index('STATE VEhICLE IS REGISTERED IN_2')])

    if(fields_values[fields_names.index('Moving_2')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('Stopped in Traffic_2')] != 'Empty'):
        data.append('1')
    else:
        data.append('-1')


    if(fields_values[fields_names.index('Pedestrian_2')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('Bicyclist_2')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('Other_2')] != 'Empty'):
        data.append('2')
        
    else:
        data.append('-1')


    data.append(fields_values[fields_names.index('NuMBER Of VEhICLES INVOLVED_2')])


    injury_death_property_info = ''
    if(fields_values[fields_names.index('Injured')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '0'
    if(fields_values[fields_names.index('Deceased')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '1'
    if(fields_values[fields_names.index('Driver')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '2'
    if(fields_values[fields_names.index('Passenger')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '3'
    if(fields_values[fields_names.index('Bicyclist_3')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '4'
    if(fields_values[fields_names.index('Proper ty')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '5'

    if(len(injury_death_property_info) == 0):
        injury_death_property_info = injury_death_property_info + '-1'
    
    data.append(injury_death_property_info)


    injury_death_property_info = ''
    if(fields_values[fields_names.index('Injured_2')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '0'
    if(fields_values[fields_names.index('Deceased_2')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '1'
    if(fields_values[fields_names.index('Driver_2')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '2'
    if(fields_values[fields_names.index('Passenger_2')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '3'
    if(fields_values[fields_names.index('Bicyclist_4')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '4'
    if(fields_values[fields_names.index('Proper ty_2')] != 'Empty'):
        injury_death_property_info = injury_death_property_info + '5'

    if(len(injury_death_property_info) == 0):
        injury_death_property_info = injury_death_property_info  + '-1'

    
    data.append(injury_death_property_info)


    if(fields_values[fields_names.index('Autonomous Mode')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('Conventional Mode')] != 'Empty'):
        data.append('1')
    else:
        data.append('-1')


    data.append(fields_values[fields_names.index('ADDRESS_2.1.0.1')])


    if(fields_values[fields_names.index('WEATHER A 1')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('WEATHER B 1')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('WEATHER C 1')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('WEATHER D 1')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('WEATHER E 1')] != 'Empty'):
        data.append('4')
    elif(fields_values[fields_names.index('WEATHER F 1')] != 'Empty'):
        data.append('5')
    elif(fields_values[fields_names.index('WEATHER G 1')] != 'Empty'):
        data.append('6')
        
    else:
        data.append('-1')


    if(fields_values[fields_names.index('WEATHER A 2')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('WEATHER B 2')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('WEATHER C 2')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('WEATHER D 2')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('WEATHER E 2')] != 'Empty'):
        data.append('4')
    elif(fields_values[fields_names.index('WEATHER F 2')] != 'Empty'):
        data.append('5')
    elif(fields_values[fields_names.index('WEATHER G 2')] != 'Empty'):
        data.append('6')
        
    else:
        data.append('-1')
    


    if(fields_values[fields_names.index('LIGHTING A 1')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('LIGHTING B 1')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('LIGHTING C 1')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('LIGHTING D 1')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('LIGHTING E 1')] != 'Empty'):
        data.append('4')
    else:
        data.append('-1')


    if(fields_values[fields_names.index('LIGHTING A 2')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('LIGHTING B 2')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('LIGHTING C 2')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('LIGHTING D 2')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('LIGHTING E 2')] != 'Empty'):
        data.append('4')
    else:
        data.append('-1')


    if(fields_values[fields_names.index('ROADWAY A 1')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('ROADWAY B 1')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('ROADWAY C 1')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('ROADWAY D 1')] != 'Empty'):
        data.append('3')
    else:
        data.append('-1')

    if(fields_values[fields_names.index('ROADWAY A 2')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('ROADWAY B 2')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('ROADWAY C 2')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('ROADWAY D 2')] != 'Empty'):
        data.append('3')
    else:
        data.append('-1')


    if(fields_values[fields_names.index('ROAD CONDITIONS A 1')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('ROAD CONDITIONS B 1')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('ROAD CONDITIONS C 1')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('ROAD CONDITIONS D 1')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('ROAD CONDITIONS E 1')] != 'Empty'):
        data.append('4')
    elif(fields_values[fields_names.index('ROAD CONDITIONS F 1')] != 'Empty'):
        data.append('5')
    elif(fields_values[fields_names.index('ROAD CONDITIONS G 1')] != 'Empty'):
        data.append('6')
    elif(fields_values[fields_names.index('ROAD CONDITIONS H 1')] != 'Empty'):
        data.append('7')
        
    else:
        data.append('-1')


    if(fields_values[fields_names.index('ROAD CONDITIONS A 2')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('ROAD CONDITIONS B 2')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('ROAD CONDITIONS C 2')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('ROAD CONDITIONS D 2')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('ROAD CONDITIONS E 2')] != 'Empty'):
        data.append('4')
    elif(fields_values[fields_names.index('ROAD CONDITIONS F 2')] != 'Empty'):
        data.append('5')
    elif(fields_values[fields_names.index('ROAD CONDITIONS G 2')] != 'Empty'):
        data.append('6')
    elif(fields_values[fields_names.index('ROAD CONDITIONS H 2')] != 'Empty'):
        data.append('7')
        
    else:
        data.append('-1')


    if(fields_values[fields_names.index('MOVEMENT A 1')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('MOVEMENT  B 1')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('MOVEMENT C 1')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('MOVEMENT  D 1')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('MOVEMENT  E 1')] != 'Empty'):
        data.append('4')
    elif(fields_values[fields_names.index('MOVEMENT  F 1')] != 'Empty'):
        data.append('5')
    elif(fields_values[fields_names.index('MOVEMENT  G 1')] != 'Empty'):
        data.append('6')
    elif(fields_values[fields_names.index('MOVEMENT  H 1')] != 'Empty'):
        data.append('7')
    elif(fields_values[fields_names.index('MOVEMENT  I 1')] != 'Empty'):
        data.append('8')
    elif(fields_values[fields_names.index('MOVEMENT J 1')] != 'Empty'):
        data.append('9')
    elif(fields_values[fields_names.index('MOVEMENT  K 1')] != 'Empty'):
        data.append('10')
    elif(fields_values[fields_names.index('MOVEMENT  L 1')] != 'Empty'):
        data.append('11')
    elif(fields_values[fields_names.index('MOVEMENT  M 1')] != 'Empty'):
        data.append('12')
    elif(fields_values[fields_names.index('MOVEMENT  N 1')] != 'Empty'):
        data.append('13')
    elif(fields_values[fields_names.index('MOVEMENT  O 1')] != 'Empty'):
        data.append('14')
    elif(fields_values[fields_names.index('MOVEMENT  P 1')] != 'Empty'):
        data.append('15')
    elif(fields_values[fields_names.index('MOVEMENT  Q 1')] != 'Empty'):
        data.append('16')
    elif(fields_values[fields_names.index('MOVEMENT  R 1')] != 'Empty'):
        data.append('17')
        
    else:
        data.append('-1')


    if(fields_values[fields_names.index('MOVEMENT A 2')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('MOVEMENT  B 2')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('MOVEMENT C 2')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('MOVEMENT  D 2')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('MOVEMENT  E 2')] != 'Empty'):
        data.append('4')
    elif(fields_values[fields_names.index('MOVEMENT  F 2')] != 'Empty'):
        data.append('5')
    elif(fields_values[fields_names.index('MOVEMENT  G 2')] != 'Empty'):
        data.append('6')
    elif(fields_values[fields_names.index('MOVEMENT  H 2')] != 'Empty'):
        data.append('7')
    elif(fields_values[fields_names.index('MOVEMENT  I 2')] != 'Empty'):
        data.append('8')
    elif(fields_values[fields_names.index('MOVEMENT J 2')] != 'Empty'):
        data.append('9')
    elif(fields_values[fields_names.index('MOVEMENT  K 2')] != 'Empty'):
        data.append('10')
    elif(fields_values[fields_names.index('MOVEMENT  L 2')] != 'Empty'):
        data.append('11')
    elif(fields_values[fields_names.index('MOVEMENT  M 2')] != 'Empty'):
        data.append('12')
    elif(fields_values[fields_names.index('MOVEMENT  N 2')] != 'Empty'):
        data.append('13')
    elif(fields_values[fields_names.index('MOVEMENT  O 2')] != 'Empty'):
        data.append('14')
    elif(fields_values[fields_names.index('MOVEMENT  P 2')] != 'Empty'):
        data.append('15')
    elif(fields_values[fields_names.index('MOVEMENT  Q 2')] != 'Empty'):
        data.append('16')
    elif(fields_values[fields_names.index('MOVEMENT  R 2')] != 'Empty'):
        data.append('17')
        
    else:
        data.append('-1')


    if(fields_values[fields_names.index('TYPE A 1')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('TYPE B 1')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('TYPE C 1')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('TYPE D 1')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('TYPE E 1')] != 'Empty'):
        data.append('4')
    elif(fields_values[fields_names.index('TYPE F 1')] != 'Empty'):
        data.append('5')
    elif(fields_values[fields_names.index('TYPE G 1')] != 'Empty'):
        data.append('6')
    elif(fields_values[fields_names.index('TYPE H 1')] != 'Empty'):
        data.append('7')
    else:
        data.append('-1')


    if(fields_values[fields_names.index('TYPE A 2')] != 'Empty'):
        data.append('0')
    elif(fields_values[fields_names.index('TYPE B 2')] != 'Empty'):
        data.append('1')
    elif(fields_values[fields_names.index('TYPE C 2')] != 'Empty'):
        data.append('2')
    elif(fields_values[fields_names.index('TYPE D 2')] != 'Empty'):
        data.append('3')
    elif(fields_values[fields_names.index('TYPE E 2')] != 'Empty'):
        data.append('4')
    elif(fields_values[fields_names.index('TYPE F 2')] != 'Empty'):
        data.append('5')
    elif(fields_values[fields_names.index('TYPE G 2')] != 'Empty'):
        data.append('6')
    elif(fields_values[fields_names.index('TYPE H 2')] != 'Empty'):
        data.append('7')
    else:
        data.append('-1')

    others = ''
    
    if(fields_values[fields_names.index('OTHER A YES')] != 'Empty'):
        others = others + '0'
    elif (fields_values[fields_names.index('OTHER A NO')] != 'Empty'):
        others = others + '1'

    
    if(fields_values[fields_names.index('OTHER B')] != 'Empty'):
        others = others + '2'
    if(fields_values[fields_names.index('OTHER C')] != 'Empty'):
        others = others + '3'
    if(fields_values[fields_names.index('OTHER D')] != 'Empty'):
        others = others + '4'
    if(fields_values[fields_names.index('OTHER E')] != 'Empty'):
        others = others + '5'
    if(fields_values[fields_names.index('OTHER F')] != 'Empty'):
        others = others + '6'
    if(fields_values[fields_names.index('OTHER G')] != 'Empty'):
        others = others + '7'
    
    if(fields_values[fields_names.index('OTHER H YES')] != 'Empty'):
        others = others + '8'
    elif (fields_values[fields_names.index('OTHER H NO')] != 'Empty'):
        others = others + '9'

    
    if(fields_values[fields_names.index('OTHER I')] != 'Empty'):
        others = others + '10'

    if(fields_values[fields_names.index('OTHER J')] != 'Empty'):
        others = others + '11'

    if(fields_values[fields_names.index('OTHER K')] != 'Empty'):
        others = others + '12'

    if(fields_values[fields_names.index('OTHER L')] != 'Empty'):
        others = others + '13'
    
    if(len(others) == 0):
        others = others + '-1'
        
    data.append(others)

    
    return data

def save_to_csv(data, csv_path):
    # Convert data to pandas DataFrame
    df = pd.DataFrame(data)
    
    # Save to CSV
    df.to_csv(csv_path, index=False)



In [6]:
#obtain the avaliable years folders 
reports_dires = os.listdir(pdf_reports_dirs)

In [7]:
crashes_lst = [] 

In [8]:
# adding the first row of the machine readable dataset as a header indicating the name of each column 
sheet_header = ['manufacturers name', 'business name','date of accident','Time of Accident','AM/PM', 'vehicle year','make','model','state vehicle is registered in','address/location of accident',
                'city','county','state','zip code','vehicle movment status','involved in the accident','number of vehicle involved','vehicle damage','Damage locations',
                'other vehicle year','other vehicle model','other vehicle state registration','other vehicle movment status','other vehicle involved in accident','other vehicle number of vehicle involved',
                'injury/death or property 1','injury/death or property 2','autonomus mode','accident decription','weather vehicle 1','weather vehicle 2','lighting vehicle 1',
                'lighting vehicle 2','road surface vehicle 1','road surface vehicle 2','road conditions vehicle 1','road conditions vehicle 2','movment preceding collision vehicle 1',
                'movment preceding collision vehicle 2','type of collsion vehcile 1','type of collsion vehcile 2','other factors']


crashes_lst.append(sheet_header)


with open(csv_file,'a', newline='') as fd:
    writer = csv.writer(fd)
    writer.writerow(sheet_header)

In [9]:
#parsing the available crashes reports and append the parsed data into a panda data frame
print(listdir(pdf_reports_dirs))
j = 0
for report_dir in reports_dires:
    reports = [f for f in listdir(pdf_reports_dirs + report_dir + '/') if isfile(join(pdf_reports_dirs + report_dir, f))]
    for report in reports:
        report_full_path = pdf_reports_dirs + report_dir + '/' + report
        form_data = extract_form_data_from_pdf(report_full_path)
        crashes_lst.append(form_data)
        form_np = np.asarray(form_data, dtype=object)
        with open(csv_file,'a', newline='') as fd:
            writer = csv.writer(fd)
            writer.writerow(form_np)
        j = j + 1

print("Done processing " + str(j) +" reports")

df = pd.DataFrame(crashes_lst)

['Collision_2019', 'Collision_2020', 'Collision_2021', 'Collision_2022', 'Collision_2023', 'Collision_2024']
Done processing 592 reports


In [11]:
#displaying some statistics from the extracted data (generated panda frame)
print(df.head(10)) 


                   0               1                 2                 3   \
0  manufacturers name   business name  date of accident  Time of Accident   
1       Aimotive, Inc  Aimotive, Inc.        09/16/2019             10:00   
2          Apple Inc.      Apple Inc.        09/19/2019              7:58   
3          Cruise LLC          Cruise        10/17/2019              1:05   
4          Cruise LLC          Cruise        10/26/2019             11:05   
5          Cruise LLC          Cruise        10/27/2019              1:50   
6          Cruise LLC          Cruise        10/28/2019              2:42   
7          Cruise LLC          Cruise        10/29/2019              7:52   
8          Cruise LLC          Cruise        11/07/2019              8:35   
9          Cruise LLC          Cruise        11/07/2019              9:10   

      4             5           6        7                               8   \
0  AM/PM  vehicle year        make    model  state vehicle is registered 

In [12]:
#avaliable fileds in the current OL316 pdf reports (as of 10/15/2024), these fileds have been used for parsing perpuses , as seen by the fitz pdf parding lib
intresting_fileds = ['MANufACTuRERS NAME', 'BuSINESS NAME', 'DATE Of ACCIDENT', 'Time of Accident', 'AM', 'PM', 'VEhICLE YEAR', 'MAkE', 'MODEL', 
                     'STATE VEhICLE IS REGISTERED IN', 'section 2  accident infoRmation.0', 'section 2  accident infoRmation.1.0', 
                     'section 2  accident infoRmation.1.1.0','section 2  accident infoRmation.1.1.1.0', 'section 2  accident infoRmation.1.1.1.1',
                    'Moving', 'Stopped in Traffic', 'Pedestrian', 'Bicyclist', 'Other', 'NuMBER Of VEhICLES INVOLVED', 'Unknown', 'None', 'minor', 'Moderate', 'major', 'click 1', 'Left Rear 1', 'Rear Bumper', 'Right Rear 1', 'Left Rear 2', 'Left Rear 3', 'Right Rear 2', 'Right Rear 3',
                    'Left Rear Passenger 1', 'Left Rear Passenger 2', 'Right Rear Passenger 1', 'Right Rear Passenger 2', 'Left Rear Passenger 3', 'Left Rear Passenger 4', 'Right Rear Passenger 3', 'Right Rear Passenger 4', 'Front Driver Side 1', 'Front Driver Side 2', 'Front Passenger Side 1', 'Front Passenger Side 2', 'Front Driver Side 3', 'Front Driver Side 4',
                    'Front Passenger Side 3', 'Front Passenger Side 4', 'Left Front Corner 1',  'Left Front Corner 2', 'Right Front Corner 1', 'Right Front Corner 2', 'Left Front Corner 3', 'Front Bumper',  'Right Front Corner 3', 'VEhICLE YEAR_2', 'MODEL_2', 'STATE VEhICLE IS REGISTERED IN_2', 'Moving_2', 'Stopped in Traffic_2', 'Pedestrian_2', 'Bicyclist_2', 'Other_2', 'NuMBER Of VEhICLES INVOLVED_2', 'Injured',  'Deceased', 'Driver', 'Passenger', 'Bicyclist_3', 'Proper ty', 'Injured_2', 'Deceased_2', 'Driver_2', 'Passenger_2', 'Bicyclist_4', 'Proper ty_2', 
                    'Autonomous Mode', 'Conventional Mode', 'ADDRESS_2.1.0.1', 
                    'WEATHER A 1', 'WEATHER A 2', 'WEATHER B 1', 'WEATHER B 2', 'WEATHER C 1', 'WEATHER C 2', 'WEATHER D 1',  'WEATHER D 2', 'WEATHER E 1', 'WEATHER E 2', 'WEATHER F 1', 'WEATHER F 2', 'WEATHER G 1', 'WEATHER G 2',
                    'LIGHTING A 1', 'LIGHTING A 2', 'LIGHTING B 1', 'LIGHTING B 2', 'LIGHTING C 1', 'LIGHTING C 2', 'LIGHTING D 1', 'LIGHTING D 2', 'LIGHTING E 1', 'LIGHTING E 2',
                    'ROADWAY A 1' ,'ROADWAY A 2' ,'ROADWAY B 1' ,'ROADWAY B 2' ,'ROADWAY C 1' ,'ROADWAY C 2' ,'ROADWAY D 1' ,'ROADWAY D 2' ,'ROAD CONDITIONS A 1' ,'ROAD CONDITIONS A 2' ,
                    'ROAD CONDITIONS B 1' ,'ROAD CONDITIONS B 2' ,'ROAD CONDITIONS C 1' ,'ROAD CONDITIONS C 2' ,'ROAD CONDITIONS D 1' ,'ROAD CONDITIONS D 2' ,'ROAD CONDITIONS E 1' ,
                    'ROAD CONDITIONS E 2' ,'ROAD CONDITIONS F 1' ,'ROAD CONDITIONS F 2' ,'ROAD CONDITIONS G 1' ,'ROAD CONDITIONS G 2' ,'ROAD CONDITIONS H 1' ,'ROAD CONDITIONS H 2',
                     'MOVEMENT A 1', 'MOVEMENT A 2' ,'MOVEMENT  B 1' ,'MOVEMENT  B 2' ,'MOVEMENT C 1' ,'MOVEMENT C 2' ,'MOVEMENT  D 1' ,'MOVEMENT  D 2' ,'MOVEMENT  E 1' ,'MOVEMENT  E 2' ,'MOVEMENT  F 1' ,'MOVEMENT  F 2' ,'MOVEMENT  G 1',
                    'MOVEMENT  G 2' ,'MOVEMENT  H 1' ,'MOVEMENT  H 2' ,'MOVEMENT  I 1' 'MOVEMENT  I 2' ,'MOVEMENT J 1' ,'MOVEMENT J 2' ,'MOVEMENT  K 1' ,'MOVEMENT  K 2' ,'MOVEMENT  L 1' ,'MOVEMENT  L 2' ,'MOVEMENT  M 1' ,'MOVEMENT  M 2' ,'MOVEMENT  N 1' ,'MOVEMENT  N 2' ,'MOVEMENT  O 1' ,'MOVEMENT  O 2' ,'MOVEMENT  P 1' ,'MOVEMENT  P 2' ,'MOVEMENT  Q 1' ,'MOVEMENT  Q 2' ,'MOVEMENT  R 1' ,'MOVEMENT  R 2',
                    'TYPE A 1' ,'TYPE A 2' ,'TYPE B 1' ,'TYPE B 2' ,'TYPE C 1' ,'TYPE C 2' ,'TYPE D 1' ,'TYPE D 2' ,'TYPE E 1' ,'TYPE E 2' ,'TYPE F 1' ,'TYPE F 2' ,'TYPE G 1' ,'TYPE G 2' ,'TYPE H 1' ,'TYPE H 2', 
                     'OTHER A YES' ,'OTHER A NO' ,'OTHER B' ,'OTHER C' ,'OTHER D' ,'OTHER E' ,'OTHER F' ,'OTHER G' ,'OTHER H YES' ,'OTHER H NO' ,'OTHER I' ,'OTHER J' ,'OTHER K' ,'OTHER L']